### 导入相关的包

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
from torchvision import datasets
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from torch.nn import TripletMarginLoss
import torch.nn.init as init
import numpy as np
import pandas as pd
from torchvision import transforms
from PIL import Image
import os
import random

### 部署模型，修改分类层

In [ ]:
# 使用预训练参数
from SqueezeNet.models.SqueezeNet import SqueezeNet

model = SqueezeNet(version="1_1")
model.load_state_dict(torch.load('squeezenet1_1_weights.pth'))


# 冻结参数层
for param in model.parameters():
    param.requires_grad = False

# 修改最后一层

print(model.classifier[1])
final_conv = nn.Conv2d(512, 256, kernel_size=1)
init.normal_(final_conv.weight, mean=0.0, std=0.01)
init.constant_(final_conv.bias, 0.0)
model.classifier[1] = final_conv


model = model.to('cuda')



for params in model.classifier[1].parameters():
    params.requires_grad = True
    
# 解冻模型的最后1个Fire模块
for param in model.features[-1:].parameters():
    param.requires_grad = True
    
    
print(model)


In [ ]:
# test_img_001_0 = Image.open('./facenet/data_cropped/001/001_0.bmp')
# test_img_001_1 = Image.open('./facenet/data_cropped/001/001_1.bmp')
# test_img_001_2 = Image.open('./facenet/data_cropped/001/001_2.bmp')
# test_img_003_3 = Image.open('./facenet/data_cropped/003/003_3.bmp')
# test_img_007_3 = Image.open('./facenet/data_cropped/007/007_3.bmp')
# test_img_003_1 = Image.open('./facenet/data_cropped/003/003_1.bmp')
# test_img_003_2 = Image.open('./facenet/data_cropped/003/003_2.bmp')

# lables = ["001_0", "001_1", "001_2", "003_3", "007_3", "003_1", "003_2"]

# transform = transforms.Compose([
#             transforms.Resize((224, 224)),
#             transforms.ToTensor(),
#             transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[
#                                  0.229, 0.224, 0.225])
#         ])

# aligned = []
# aligned.append(transform(test_img_001_0))
# aligned.append(transform(test_img_001_1))
# aligned.append(transform(test_img_001_2))
# aligned.append(transform(test_img_003_3))
# aligned.append(transform(test_img_007_3))
# aligned.append(transform(test_img_003_1))
# aligned.append(transform(test_img_003_2))

# model.eval()
# aligned = torch.stack(aligned).to('cuda')
# emdeddings = model(aligned).cpu().detach()

# dists = [[(e1 - e2).norm().item() for e2 in emdeddings] for e1 in emdeddings]
# pd.DataFrame(dists, columns=lables, index=lables)




### 读取并且定制数据集

In [ ]:
loss = TripletMarginLoss(margin=0.2, p=2)


# 分割三元组数据集
class TripletFaceDataset(Dataset):
    def __init__(self, image_folder):
        self.image_folder = image_folder
        self.persons = os.listdir(image_folder)

    def __getitem__(self, _):
        anchor_person = random.choice(self.persons)
        positive_person = anchor_person
        negative_person = random.choice(self.persons)
        while negative_person == anchor_person:
            negative_person = random.choice(self.persons)

        anchor_img_path = random.choice(os.listdir(
            os.path.join(self.image_folder, anchor_person)))
        positive_img_path = random.choice(os.listdir(
            os.path.join(self.image_folder, positive_person)))
        negative_img_path = random.choice(os.listdir(
            os.path.join(self.image_folder, negative_person)))

        anchor_img = Image.open(os.path.join(
            self.image_folder, anchor_person, anchor_img_path))
        positive_img = Image.open(os.path.join(
            self.image_folder, anchor_person, positive_img_path))
        negative_img = Image.open(os.path.join(
            self.image_folder, positive_person, negative_img_path))

        # 进行数据处理
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[
                                 0.229, 0.224, 0.225])
        ])

        anchor_img = transform(anchor_img)
        positive_img = transform(positive_img)
        negative_img = transform(negative_img)

        return anchor_img, positive_img, negative_img

    def __len__(self):
        return len(self.persons) * 5  # 假设每个人有5张图片

labels = os.listdir('./facenet/data_cropped')
# print(labels)

# 切割数据集八二分

batch_size = 64
epochs = 800
workers = 0 if os.name == 'nt' else 8

transforming = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[
                                 0.229, 0.224, 0.225])
        ])
dataset = datasets.ImageFolder('./facenet/data_cropped', transform=transforming)

img_inds = np.arange(len(labels))
np.random.shuffle(img_inds)
train_inds = img_inds[:int(0.8 * len(img_inds))]
valid_inds = img_inds[int(0.8 * len(img_inds)):]

train_loaders = DataLoader(
    dataset, batch_size = batch_size, sampler = train_inds)

valid_loaders = DataLoader(
    dataset, batch_size = batch_size, sampler = valid_inds)


### 开始模型训练

In [ ]:
# 定制优化器
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)